### Testing custom package

In [2]:
from nav_pii_anon.spacy.spacy_model import SpacyModel

In [3]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652. "
      "Telefonnummeret mitt er 94867658 og jeg reiser "
      "hjem 24.07.2020")

In [4]:
nlp = SpacyModel()

In [5]:
graph = nlp.dependency_graph(tx)

In [6]:
graph.nodes

NodeView(('navn', 'mitt', 'marius', 'er', 'personnummer', 'og', '15044216652.', 'telefonnummeret', '94867658', 'reiser', 'jeg', '24.07.2020', 'hjem'))

In [7]:
graph.edges

EdgeView([('navn', 'mitt'), ('navn', 'marius'), ('mitt', 'personnummer'), ('mitt', 'telefonnummeret'), ('marius', 'er'), ('marius', 'personnummer'), ('er', '15044216652.'), ('er', '94867658'), ('personnummer', 'og'), ('og', 'reiser'), ('15044216652.', 'telefonnummeret'), ('telefonnummeret', '94867658'), ('94867658', 'reiser'), ('reiser', 'jeg'), ('reiser', '24.07.2020'), ('24.07.2020', 'hjem')])

In [4]:
nlp = SpacyModel(spacy.load("nb_core_news_lg"))
reg = SpacyModel(spacy.load("nb_core_news_lg"))

In [5]:
tx = "Mitt navn er Marius og mitt personnummer er 15044216652"

In [6]:
nlp.predict(tx)

[['Marius', 'PER', 3, 4, 'NA']]


In [7]:
nlp.add_patterns()
reg.add_patterns()

In [8]:
nlp.predict(tx)

(15044216652,)
[['Marius', 'PER', 3, 4, 'NA'], ['15044216652', 'FNR', 8, 9, 'NA']]


In [9]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652."
      "Telefonnummeret mitt er 94867658 og jeg reiser"
      "hjem 19.07.2020")

In [10]:
nlp.predict(tx)

(94867658,)
(94867658, 19.07.2020)
[['Marius', 'PER', 3, 4, 'NA'], ['94867658', 'TLF', 11, 12, 'NA'], ['reiserhjem', 'ORG', 14, 15, 'NA'], ['19.07.2020', 'DTM', 15, 16, 'NA']]


In [ ]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652. "
      "Telefonnummeret mitt er 94867658 og jeg reiser "
      "hjem 24.07.2020")

In [ ]:
nlp.predict(tx)

In [ ]:
reg.disable_NER()

In [ ]:
reg.predict(tx)

In [ ]:
reg.enable_NER()

In [ ]:
reg.predict(tx)

In [ ]:
nlp.display_predictions(tx)

In [ ]:
print(nlp.replace(tx))